# Trace Upstream Script 
## Version 2: Directly create upstream trib feature classes 
Kernel: Python 2

Developed by: Curtis Fang

In [1]:
import arcpy
import numpy as np
import pandas as pd

### Set GDB directory

In [17]:
# arcpy.env.workspace='P:\GIS\Projects\LARiver_Watershed_Projects\General\Shapefiles\Subwatersheds\SubWSTributary.gdb'
arcpy.env.workspace='C:\Users\cfang\Documents\GitHub\TraceUpstream\sandbox.gdb'
input_data='LAC_catchment'

### Import input feature class as a pandas dataframe 

In [18]:
arcpy.conversion.TableToExcel(Input_Table=input_data,
                              Output_Excel_File='Output/input.xls'
                             )
df = pd.read_excel('output/input.xls')

In [19]:
orig_cols=df.columns
orig_cols

Index([u'OBJECTID_1', u'OBJECTID', u'NAME', u'NAME2', u'WATERSHED', u'AREA_AC',
       u'NEWMC', u'IMPCOVMC', u'ICONFIGMC', u'ROADDMC', u'SLOPEMC',
       u'COMBINEDMC', u'MC_ID', u'WATERSHE_1', u'NEW_WSID', u'NEW_DS_ID',
       u'RECCOMEN_1', u'PREC_97_06', u'PREC_GAGE', u'PRE_97_06', u'PEVT_87_06',
       u'DUMMY', u'IRRIGATION', u'OLD_MC', u'NEW_MC', u'FINAL_MC', u'SAME_MC',
       u'IMP_COVER', u'IMP_CONFIG', u'ROAD_DENS', u'SLOPE_MC', u'SURO_INAC',
       u'PERO_INAC', u'RCH_GRP', u'Shape_Leng', u'Shape_Le_1', u'DEPTH_RATI',
       u'Shape_Length', u'Shape_Area', u'CatID', u'DwnCatID'],
      dtype='object')

### Set target downstream catchment and catchment ID columns 

In [20]:
#Input
target_ID='c6475'
id_col='CatID'
ds_col='DwnCatID'

### Define output variables

In [21]:
temp_input="_temp_input_"+target_ID
temp_input_xls='Output/'+temp_input+'.xls'
table_gdb='a'+target_ID+'_upstream'
table_csv=filename = 'Output/'+table_gdb+'.csv'
fc_output='upstream_'+target_ID

### Trace upstream

In [22]:
_neighbors = df[df[ds_col] == target_ID]
upstream = _neighbors
while not _neighbors.empty:
    _neighbors = df[df[ds_col].isin(_neighbors[id_col])]
    upstream = upstream.append(_neighbors, ignore_index=True)
#Comment out the line below if one does not want to include the catchment itself
upstream=upstream.append(df[df[id_col]==target_ID],ignore_index=True)

In [23]:
exc_cols=orig_cols.difference([id_col])
upstream = upstream.drop(exc_cols,1)
upstream

,CatID
0,c6476
1,c6477
2,c6478
3,c6482
4,c6479
5,c6480
6,c6483
7,c6491
8,c6481
9,c6484


In [24]:
upstream.to_csv(table_csv,index=False)
arcpy.conversion.TableToTable(table_csv,
                             arcpy.env.workspace,
                            table_gdb)

<Result 'C:\\Users\\cfang\\Documents\\GitHub\\TraceUpstream\\sandbox.gdb\\ac6475_upstream'>

In [25]:
#Create a temp duplicate shapefile in prep for join field - avoid modifying the original input
arcpy.conversion.FeatureClassToFeatureClass(input_data, 
                                            arcpy.env.workspace, 
                                            temp_input)

<Result 'C:\\Users\\cfang\\Documents\\GitHub\\TraceUpstream\\sandbox.gdb\\_temp_input_c6475'>

In [26]:
arcpy.management.JoinField(in_data=temp_input,
                          in_field=id_col,
                          join_table= table_gdb,
                          join_field=id_col)

<Result 'C:\\Users\\cfang\\Documents\\GitHub\\TraceUpstream\\sandbox.gdb\\_temp_input_c6475'>

In [27]:
# Check join result
arcpy.conversion.TableToExcel(Input_Table=temp_input,Output_Excel_File=temp_input_xls)
df3 = pd.read_excel(temp_input_xls)
df3

,OBJECTID_1,OBJECTID,NAME,NAME2,WATERSHED,AREA_AC,NEWMC,IMPCOVMC,ICONFIGMC,ROADDMC,...,PERO_INAC,RCH_GRP,Shape_Leng,Shape_Le_1,DEPTH_RATI,CatID,DwnCatID,Shape_Length,Shape_Area,CatID_1
0,1,1,BI57-8,BI57-9,ballona,1479.189941,1101,1,1,0,...,10.57,1100,70406.763151,70406.763489,0,c1103,c1102,70406.763489,6.443331e+07,NaN
1,2,2,BI57-7,BI57-8,ballona,1535.420044,1101,1,1,0,...,10.22,1100,49957.756595,49957.756660,0,c1104,c1103,49957.756660,6.688276e+07,NaN
2,3,3,STONE1,STONE3,ballona,1489.010010,1001,1,0,0,...,7.25,1043,67077.347428,67077.347040,0,c1044,c1043,67077.347040,6.486132e+07,NaN
3,4,4,BNDCT1,BNDCT2,ballona,1749.709961,1001,1,0,0,...,8.26,1054,46582.212354,46582.212581,0,c1055,c1054,46582.212581,7.621715e+07,NaN
4,5,5,SEPUL1,SEPUL2,ballona,2210.479980,1001,1,0,0,...,6.04,1037,60002.639989,60002.639968,0,c1038,c1037,60002.639969,9.628830e+07,NaN
5,6,6,MNDVL1,SNMON1,ballona,2336.600098,1,0,0,0,...,4.00,1116,60667.453241,60667.453070,0,c1117,c1116,60667.453070,1.017822e+08,NaN
6,7,7,BI497-1,BI497-2,ballona,1004.309998,1001,1,0,0,...,10.07,1033,52963.412473,52963.412566,0,c1035,c1034,52963.412566,4.374775e+07,NaN
7,8,8,MORGA1,SEPUL2,ballona,938.966980,1001,1,0,0,...,7.52,1037,51007.460107,51007.459708,0,c1039,c1037,51007.459708,4.090140e+07,NaN
8,9,9,RUSTC1,RUSTC2,ballona,1482.829956,1,0,0,0,...,3.05,1115,36558.229520,36558.229656,0,c1122,c1121,36558.229656,6.459211e+07,NaN
9,10,10,BI57-1,BI57-5,ballona,1184.619995,1100,1,1,0,...,10.43,1110,67789.474647,67789.474485,0,c1111,c1110,67789.474485,5.160205e+07,NaN


In [28]:
expression = id_col+'_1 IS NOT NULL' #Furture improvement: Avoid hard coding the epxression
arcpy.conversion.FeatureClassToFeatureClass(temp_input, 
                                            arcpy.env.workspace, 
                                            fc_output,
                                           expression)

<Result 'C:\\Users\\cfang\\Documents\\GitHub\\TraceUpstream\\sandbox.gdb\\upstream_c6475'>

In [29]:
#if run into already exist error, run this block
#arcpy.management.Delete(fc_output)
# arcpy.env.overwriteOutput = True

### Clean up temp files 

In [30]:
arcpy.management.Delete(temp_input)
arcpy.management.DeleteField(fc_output,id_col+'_1')

<Result 'C:\\Users\\cfang\\Documents\\GitHub\\TraceUpstream\\sandbox.gdb\\upstream_c6475'>